# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [3]:
import os, re, glob
import h5py
import btrack
import napari
import numpy as np
import pandas as pd
from skimage import io, filters, util, segmentation, morphology, measure, restoration, exposure
from scipy import stats, spatial, ndimage

ModuleNotFoundError: No module named 'napari'

In [5]:
pipeline_name = "CBn"
plateID = "AU03501"
well = "A1"
field = "field_1"
field_num = field.replace("field_","")
ch1_name = 'NR'
ch2_name = 'CC'
data_path = '/home/exacloud/gscratch/HeiserLab/images/'
#data_path = '/Users/dane/Documents/CellTrackingProjects/AU565/images/'
int_path = os.path.join(data_path+plateID,"Analysis",pipeline_name,"intermediate_files/")

reg_filename = os.path.join(int_path,plateID+"_R_"+well+"_"+field_num+"_reg_stack.tif")
mask_filename = reg_filename.replace("_reg_stack.tif", "_nuc_masks_stack.png")
tracking_path = os.path.join(int_path,'tracking/')
if not os.path.exists(tracking_path+well+"/"+field+"/nuc_masks/"):
        os.makedirs(tracking_path+well+"/"+field+"/nuc_masks/")
if not os.path.exists(tracking_path+well+"/"+field+"/reg"):
        os.makedirs(tracking_path+well+"/"+field+"/reg")
if not os.path.exists(tracking_path+well+"/"+field+"/results"):
        os.makedirs(tracking_path+well+"/"+field+"/results")
if not os.path.exists(tracking_path+well+"/"+field+"/filtered_masks"):
        os.makedirs(tracking_path+well+"/"+field+"/filtered_masks")

In [6]:
first_frame = 8
last_frame = 18
nuc_mask_file_names = glob.glob(os.path.join(int_path, 'tracking', well,field, 'nuc_masks','mask*.tif'))[first_frame:last_frame]
nuc_reporter_file_names = glob.glob(os.path.join(int_path, 'tracking', well, field, 'reg','t*.tif'))[first_frame:last_frame]

# sort the files numerically
#files = sorted(files, key=lambda f: int(f[len(os.path.join(int_path, 'labels_')):-4]))
nuc_mask_file_names = sorted(nuc_mask_file_names)
nuc_reporter_file_names = sorted(nuc_reporter_file_names)

nuc_reporter_stack_filename = glob.glob(os.path.join(int_path, plateID+"_R_"+well+"_"+field_num+"_reg_stack.tif"))[0]
nuc_reporter_stack = io.imread(nuc_reporter_stack_filename)[first_frame:last_frame,:,:]

phase_stack_filename = glob.glob(os.path.join(int_path, plateID+"_P_"+well+"_"+field_num+"_reg_stack.tif"))[0]
phase_stack = io.imread(phase_stack_filename)[first_frame:last_frame,:,:]

nuc_reporter_stack_filename
#nuc_reporter_stack.shape
#len(nuc_reporter_file_names)



IndexError: list index out of range

### method 1 - using a numpy array

In this example, each image from the timelapse is a 3D volume (32 x 1200 x 1200) and there are 10 timepoints

In [ ]:
def segmentation_arr(files):
    """Segmentation as numpy array."""
    
    stack = []
    for filename in files:
        img = io.imread(filename)
        stack.append(img)
    return np.stack(stack, axis=0)

nuc_mask_stack = segmentation_arr(nuc_mask_file_names)


In [ ]:
stack = segmentation_arr(files)

Now we print out the shape of the stack (T, Z, Y, X):

In [ ]:
stack.shape

### method 2 - using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format. Note that the generator produces a numpy array for each image...

In [7]:
def segmentation_generator(files):
    """Segmentation generator"""
    
    for filename in files:
        img = io.imread(filename)
        yield img

nuc_mask_generator = segmentation_generator(nuc_mask_file_names)
nuc_reporter_generator = segmentation_generator(nuc_reporter_file_names)



## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 2x). Note that we can also use scikit-image `regionprops` to calculate properties for each object, using the `properties` keyword:

obj_from_arr = btrack.utils.segmentation_to_objects(stack, properties=('area', ))

# inspect the first object
obj_from_arr[0]

In [8]:
obj_from_generator = btrack.utils.segmentation_to_objects(
    nuc_mask_generator, 
    intensity_image = nuc_reporter_generator,
    properties = ('area', 'major_axis_length')
)



...Found no objects.


In [9]:
# inspect the first object
obj_from_generator[0]

IndexError: list index out of range

## run btrack with the objects

We will use the objects from the generator here.

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config_AU565.json')
    tracker.max_search_radius = 100

    # append the objects to be tracked
    tracker.append(obj_from_generator)

    # set the volume
    tracker.volume=((0, nuc_mask_stack.shape[2]), (0, nuc_mask_stack.shape[1]), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=200)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(int_path, "tracking",well,field,"results",'tracking.h5'), obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=2)
    
    #tracks = tracker.tracks

## visualize with napari

Note that we also set the scale of the images here to account for the anisotropy.

#viewer = napari.Viewer()
#viewer = napari.view_image(img_p_stack, name = "phase image")

viewer = napari.view_image(phase_stack, name = "phase image")
viewer.add_labels(stack,  name='Segmentation')
viewer.add_tracks(data, properties=properties, graph=graph, name='Tracks')

napari.run()



Use the graph results to asign progeny and ancestors

graph_filename = os.path.join(int_path, "tracking",well,field,"results","graph.csv") graph_df = pd.DataFrame.from_dict(graph,orient='index', columns=['parent']) graph_df.reset_index(inplace=True) graph_df.rename(columns = {'index':'label'}) graph_df.to_csv(graph_filename,index=False)

 

Create a new file tracks.csv with the following columns:
label - a unique label of the track (label of markers, 16-bit positive value)
begins - a zero-based temporal index of the frame in which the track begins
ends - a zero-based temporal index of the frame in which the track ends
parent - label of the parent track (0 is used when no parent is defined)
length - The number of frames that the cell is identified in
plateID - Character string of the plate's ID such as AU02001
well - Character string of the well such as A1
field - Integer of the image field within the well

    """Return an LBEP list describing the track lineage information.
    Notes
    -----
    L : int
        A unique label of the track (label of markers, 16-bit positive).
    B : int
        A zero-based temporal index of the frame in which the track begins.
    E : int
        A zero-based temporal index of the frame in which the track ends.
    P : int
        Label of the parent track (0 is used when no parent is defined).
    R : int
        Label of the root track.
    G : int
        Generational depth (from root).



In [ ]:
tracking_filename = os.path.join(int_path,"tracking",well,field,"results","tracking.h5")
tracking = h5py.File(tracking_filename, 'r')
lbepr = tracking_tracks['obj_type_1']['LBEPR'][:]

In [ ]:
#set filter parameters
min_track_length = 3

tracking_filename = os.path.join(int_path,"tracking",well,field,"results","tracking.h5")
res_flt_filename = tracking_filename.replace("tracking.h5","tracks.csv")
tracking = h5py.File(tracking_filename, 'r')
lbepr = tracking_tracks['obj_type_1']['LBEPR'][:]
lbepr_df = pd.DataFrame(lbepr, columns=['label','begins','ends','parent','root','depth'])
lbepr_df['length'] = lbepr_df.ends - lbepr_df.begins + 1
last_track = lbepr_df.ends.max()
#check if object is a parent
lbepr_df["is_parent"] = lbepr_df['label'].isin(lbepr_df['parent'])
lbepr_df['plateID'] = plateID
lbepr_df['well'] = well
lbepr_df['field'] = field.replace("field_","")
#If filtered results do not exist, read in the res_track.txt file for the current field
if not os.path.exists(res_flt_filename):
    #Filter using the filter parameters
    #remove short tracks that are not parents and are not in the last frame
    tracks_flt = lbepr_df.query('length >= @min_track_length or is_parent or ends > (@last_track-@min_track_length)')
    ##remove any track that appears after the first frame and doesn't have a parent
    #tracks_flt = tracks_flt.query('not (begins > 1 & parent == 0)')
    #write out the res_flt_track.txt file
    tracks_flt.to_csv(res_flt_filename,index=False) 

In [ ]:
mask_track_path = os.path.join(int_path,"tracking",well,field,"results")
tracked_mask_filenames = sorted(glob.glob(mask_track_path+"/mask*"))[first_frame:last_frame]
#condition on whether the filtered masks exist
if not os.path.exists(tracked_mask_filenames[0].replace("results","filtered_masks")):
   #read in the tracks file for this field    
    res_flt_filename = os.path.join(int_path,"tracking",well,field,"results","tracks.csv")
    tracks = pd.read_csv(res_flt_filename) 
    #loop through the mask images in the field
    for fn in tracked_mask_filenames:
        #read in the mask image
        im = io.imread(fn)
        #replace any label that's not a cell with a 0 value
        cell_labels = np.array([x if x in tracks.label.to_numpy()
                                   else 0 for x in range(0, im.max()+1)])
        im_filtered = cell_labels[im]
        io.imsave(fn.replace("results","filtered_masks"), im_filtered.astype(np.int16), plugin='tifffile', check_contrast=False)


In [ ]:
#If the metadata exists, load it
metadata_filename = os.path.join(data_path,plateID,"metadata",plateID+".xlsx")

if os.path.exists(metadata_filename):
    md_all = pd.read_excel(metadata_filename, engine='openpyxl', dtype={'Drug1Concentration': str, 'Drug2Concentration': str})
    
    #remove unwanted columns read in from the excel files
    r = re.compile("Unnamed.*")
    columns_to_drop = list(filter(r.match, md_all.columns)) 
    metadata = md_all.drop(columns = columns_to_drop)
    
    #match metadata and data well labels format
    metadata['row'] = [re.sub(r'[0-9]*', '', Well) for Well in metadata['Well']]
    metadata['column'] = [re.sub(r'[A-Z]', '', Well) for Well in metadata['Well']]
    metadata['column'] = [re.sub(r'\A0', '', row) for row in metadata['column']]
    metadata['well'] = metadata['row'] + metadata['column']
    



In [ ]:
cyto_expansion = 5
minutes_between_images = 30
g1_threshold = .94
neighborhood_nuclei_distance = 5
neighborhood_radius_near = 20
neighborhood_radius_medium = 45
neighborhood_radius_far = 70

def calc_G1_prop(x):
    G1_count = x.value_counts()[1] #G1
    G1_prop = G1_count/len(x)
    return G1_prop

l0_filename = os.path.join(data_path+plateID,"Analysis",pipeline_name,plateID+"_"+well+"_"+field+"_level_0.csv")

#condition on whether the l1 file exists
#if not os.path.exists(l0_filename.replace('level_0','level_1')):
print("Pulling data from images "+l0_filename.replace('_level_0.csv',''))
filtered_mask_path = os.path.join(int_path,"tracking",well,field,"filtered_masks")
tracked_mask_filenames = sorted(glob.glob(filtered_mask_path+"/mask*"))[first_frame:last_frame]
img_gs_reg = io.imread(int_path+plateID+"_G_"+well+"_"+field.replace("field_","")+"_reg_stack.tif")[first_frame:last_frame]
# iterate over the mask files
results = []
for i, fn in enumerate(tracked_mask_filenames):
    #read in the mask image
    masks = io.imread(fn)
    #read in registered R images
    reg_fn = fn.replace("filtered_masks","reg")
    image = io.imread(reg_fn.replace("mask","t"))

    #measure reporter intensity and nuclear morphology, texture
    nuclei = measure.regionprops_table(masks, intensity_image=image,
                                       properties=('label',
                                                   'area','bbox_area','convex_area','centroid','eccentricity','equivalent_diameter','extent','feret_diameter_max','filled_area',
                                                    'major_axis_length','minor_axis_length','moments_hu','perimeter','perimeter_crofton','solidity',
                                                    'mean_intensity','max_intensity','min_intensity'))
    #expand the masks to get cytoplasmic regions
    nuclei_boundaries = segmentation.find_boundaries(masks, mode='thick')*masks
    nuclei_expansions = segmentation.expand_labels(masks, cyto_expansion) - masks + nuclei_boundaries

    # measure nuclear and cytoplasmic intensities and textures in the green channel
    nuclei_g = measure.regionprops_table(masks, intensity_image=img_gs_reg[i],
                                         properties=('label','centroid',
                                                     'mean_intensity','max_intensity','min_intensity'))
    nuclei_exp_g = measure.regionprops_table(nuclei_expansions, intensity_image=img_gs_reg[i],
                                             properties=('label','centroid',
                                                         'mean_intensity','max_intensity','min_intensity'))

    # turn results into a dataframe
    nuclei_data = pd.DataFrame(nuclei)
    nuclei_data.rename(columns={col: 'Nuclei_'+pipeline_name+'_' +ch1_name+'_'+col  for col in nuclei_data.columns if col not in ['label']}, inplace=True)

    nuclei_g_data = pd.DataFrame(nuclei_g)
    nuclei_g_data.rename(columns={col: 'Nuclei_'+pipeline_name+'_' +ch2_name+'_'+col  for col in nuclei_g_data.columns if col not in ['label']}, inplace=True)

    nuclei_exp_g_data = pd.DataFrame(nuclei_exp_g)
    nuclei_exp_g_data.rename(columns={col: 'Cyto_'+pipeline_name+'_' +ch2_name+'_'+col  for col in nuclei_exp_g_data.columns if col not in ['label']}, inplace=True)

    # recover the well and field values and add them to the dataframe
    well = re.findall('/[A-Z][0-9]+/',reg_fn)[0]
    well = re.sub('/','', well)
    nuclei_data['well'] = well
    field = re.findall('field_[0-9]+',reg_fn)[0]
    field = int(re.sub('field_','', field))
    nuclei_data['field'] = field
    nuclei_data['slice'] = i
    nuclei_data['elapsed_minutes'] = i*minutes_between_images #assumes time slice numbering starts at 1
    elapsed_minutes = i*minutes_between_images #assumes time slice numbering starts at 1
    day = np.floor(elapsed_minutes/(24*60)).astype(int)
    hour = np.floor((elapsed_minutes-day*(24*60))/60).astype(int)
    minute = np.floor(elapsed_minutes-day*(24*60)-hour*60).astype(int)
    day = str(day).zfill(2)
    hour = str(hour).zfill(2)
    minute = str(minute).zfill(2)
    nuclei_data['time_slice'] = day+"d"+hour+"h"+minute+"m"

    #Calculate ratio of ch2 cyto to nuclei intensities
    nuclei_exp_g_data['Cell_'+pipeline_name+'_' +ch2_name+'_mean_intensity_ratio'] = nuclei_exp_g_data['Cyto_'+pipeline_name+'_' +ch2_name+'_mean_intensity']/nuclei_g_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_mean_intensity']
    nuclei_exp_g_data['Cell_'+pipeline_name+'_' +ch2_name+'_max_intensity_ratio'] = nuclei_exp_g_data['Cyto_'+pipeline_name+'_' +ch2_name+'_max_intensity']/nuclei_g_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_max_intensity']
    nuclei_exp_g_data['Cell_'+pipeline_name+'_' +ch2_name+'_min_intensity_ratio'] = nuclei_exp_g_data['Cyto_'+pipeline_name+'_' +ch2_name+'_min_intensity']/nuclei_g_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_min_intensity']

    #label cell states based on the reporter ratio
    nuclei_exp_g_data['cell_cycle_state'] = 'G1'
    mask = nuclei_exp_g_data['Cell_'+pipeline_name+'_' +ch2_name+'_mean_intensity_ratio'] > g1_threshold
    nuclei_exp_g_data.loc[mask, 'cell_cycle_state'] = 'S/G2'
    nuclei_exp_g_data['cell_cycle_state_threshold'] = g1_threshold

    #calculate the neighborhood density
    nuclei_kd = spatial.KDTree(nuclei_data[['Nuclei_'+pipeline_name+'_NR_centroid-0','Nuclei_'+pipeline_name+'_NR_centroid-1']])
    nuclei_data['neighborhood_'+str(neighborhood_radius_near)] = nuclei_kd.query_ball_point(nuclei_data[['Nuclei_'+pipeline_name+'_NR_centroid-0','Nuclei_'+pipeline_name+'_NR_centroid-1']],
                                                                                            r = neighborhood_radius_near, return_sorted = True, return_length=True)
    nuclei_data['neighborhood_'+str(neighborhood_radius_medium)] = nuclei_kd.query_ball_point(nuclei_data[['Nuclei_'+pipeline_name+'_NR_centroid-0','Nuclei_'+pipeline_name+'_NR_centroid-1']],
                                                                                            r = neighborhood_radius_medium, return_sorted = True, return_length=True)
    nuclei_data['neighborhood_'+str(neighborhood_radius_far)] = nuclei_kd.query_ball_point(nuclei_data[['Nuclei_'+pipeline_name+'_NR_centroid-0','Nuclei_'+pipeline_name+'_NR_centroid-1']],
                                                                                            r = neighborhood_radius_far, return_sorted = True, return_length=True)

    #merge the dataframes from the different channels
    df = pd.merge(nuclei_data, nuclei_g_data, how="left", on=["label"])
    df_all = pd.merge(df, nuclei_exp_g_data, how="left", on=["label"])

    # append this image's dataframe to the results list
    results.append(df_all)

#concatenate all of the results from all images in the field
l0_image = pd.concat(results)

#join with the tracking results to get lineage, parent, frame length values
tracks_filename = os.path.join(int_path,"tracking",well,"field_"+str(field),"results/tracks.csv")
tracks = pd.read_csv(tracks_filename)
l0 = pd.merge(l0_image, tracks, how="left", on=["label", "well", "field"])

#label cell states based on the reporter ratio
l0['cell_cycle_state'] = 'G1'
mask = l0['Cell_'+pipeline_name+'_CC_mean_intensity_ratio'] > g1_threshold
l0.loc[mask, 'cell_cycle_state'] = 'S/G2'

if os.path.exists(metadata_filename):
    #merge data and metadata on well values
    l1= pd.merge(l0, metadata, how="left", on=["well"]).round(decimals=2)
    l1['treatment'] =  l1['Drug1']+'_'+l1['Drug1Concentration']+'_'+l1['Drug2']+'_'+l1['Drug2Concentration']
    #filter out first 9 slices
#        l1 = l1[l1['slice'] > 9]
    print("Writing "+l0_filename.replace('level_0','level_1') + " to disk")
    l1.to_csv(l0_filename.replace('level_0','level_1'), index = False)
    #summarize to image level
    l2 = l1.groupby(['plateID','well','field', 'time_slice', 'elapsed_minutes','Drug1','Drug1Concentration', 'Drug2', 'Drug2Concentration', 'treatment', 'cell_cycle_state_threshold']).agg(
        n = ('plateID', 'size'),
        length=('length', 'mean'),
        Nuclei_CBn_NR_area=('Nuclei_CBn_NR_area', 'mean'),
        Nuclei_CBn_CC_mean_intensity=('Nuclei_CBn_CC_mean_intensity', 'mean'),
        Cyto_CBn_CC_mean_intensity=('Cyto_CBn_CC_mean_intensity', 'mean'),
        Cell_CBn_CC_mean_intensity_ratio=('Cell_CBn_CC_mean_intensity_ratio', 'mean'),
        Cell_CBn_CC_max_intensity_ratio=('Cell_CBn_CC_max_intensity_ratio', 'mean'),
        Cell_CBn_CC_min_intensity_ratio=('Cell_CBn_CC_min_intensity_ratio', 'mean'),
#            G1_proportion =('cell_cycle_state', calc_G1_prop),
        neighborhood_20 = ('neighborhood_'+str(neighborhood_radius_near), 'mean'), #TODO figure out how to summarize with dynamic name
        neighborhood_45 = ('neighborhood_'+str(neighborhood_radius_medium), 'mean'),
        neighborhood_70 = ('neighborhood_'+str(neighborhood_radius_far), 'mean')
    ).reset_index().round(decimals=2)
    l2['cell_cycle_state_threshold'] = g1_threshold

    print("Writing "+l0_filename.replace('level_0','level_2') + " to disk")
    l2.to_csv(l0_filename.replace('level_0','level_2'), index = False)
else:
    print("no metadata file for "+plateID+" so creating level 0 file")
    l0 = l0.round(decimals=2)
    l0.to_csv(l0_filename, index = False)
